In [18]:
import pandas as pd
import dask.dataframe as dd
from dask.multiprocessing import get

import itertools
pd.set_option('display.max_rows', 700)
pd.set_option('display.max_columns', 600)

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model, svm, tree
from sklearn.ensemble import RandomForestRegressor
from  sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
sns.set(rc={'figure.figsize':(16,9)})
#import swifter

np.random.seed(1907)

In [19]:
from tqdm import tqdm
# from tqdm.auto import tqdm  # for notebooks

#tqdm.pandas()

In [20]:
def initial_data_preparation():
    

    data = pd.read_csv("spark_processed_data.csv", sep="|")

    
    
    print(f"{len(data)} Rows.")
   

    multi_choice_columns = pd.read_csv("spark_multi_choice_columns.csv", sep="|")["0"].tolist()
    
    data = data.drop(multi_choice_columns, axis=1)
    
    # Preserve country columns for target transformation
    data["Country_for_target_transformation"] = data["Country"]
    
    
    # One hot encoding
    cat_columns = ["Country", "Gender", "JobSat", 
                                        "JobSeek",
                                        "Employment",
                                        "MainBranch", 
                                        "Hobbyist", 
                                        "EdLevel", 
                                        "NEWDevOps", 
                                        "NEWDevOpsImpt", 
                                        "NEWEdImpt", 
                                        "NEWLearn",
                                        "NEWOffTopic",
                                        "NEWOnboardGood",
                                        "NEWOtherComms",
                                        "NEWOvertime",
                                        "NEWPurpleLink","OpSys","OrgSize",
                                        "PurchaseWhat","SOAccount",
                                        "SOComm","SOPartFreq","SOVisitFreq",
                                        "SurveyEase", "SurveyLength","Trans", "UndergradMajor","WelcomeChange"]
    
    cat_columns = [col for col in cat_columns if col not in multi_choice_columns]
    data = pd.get_dummies(data=data, columns=cat_columns)

    
    data = data.dropna(subset=["ConvertedComp"], axis=0)
    data = data[data["ConvertedComp"] >0]
    
    #Drop countries that has less than 150 observation
    countries = data["Country_for_target_transformation"].value_counts()
    countries_to_drop = countries[countries <= 200].index.tolist()
    data = data[~data["Country_for_target_transformation"].isin(countries_to_drop)].reset_index(drop=True)
    print(f"{len(data)} Rows.")
    
    
    # Fill NaNs with mean
    nas = data.isna().sum()
    nas = nas[nas > 0]

    for na_col in nas.index:
        data[na_col].fillna(data[na_col].mean(), inplace=True)

    #LABEL :"ConvertedComp"
    return data

In [21]:
data = initial_data_preparation()

64461 Rows.
30179 Rows.


## TODO
- How many technologies he/she would like to learn? (DatabaseDesireNextYear, LanguageDesireNextYear) - Erdem **DONE**
- How many technologies was used in the past? (DatabaseWorkedWith, LanguageWorkedWith) - Erdem **DONE**
- Interaction between Job Satisfaction & Job Seeking columns and their impact on  compansation - Berk
- New Job Hunt:Better Comp. & Job Seeking. - Berk
- Target Encoding - Country:Done Target Transformation, Tech, Currency  - **Inprogress**

#### How many options are chosen in a given option set?

#### High leverage and low leveraged tech

In [22]:
def target_transformation(x_train, x_test, y_train, y_test):
    train_set = pd.concat([x_train, y_train], axis=1)
    test_set = pd.concat([x_test, y_test], axis=1)
    
    avg_prf_comp_by_country = train_set.groupby("Country_for_target_transformation").ConvertedComp.mean()
    target_transformer = lambda row: (row['ConvertedComp'] / avg_prf_comp_by_country[row["Country_for_target_transformation"]]) *100
    
    y_train = train_set.apply(target_transformer, axis=1 ).rename("ConvertedComp_index")
    y_test = test_set.apply(target_transformer, axis=1 ).rename("ConvertedComp_index")
     
    
    return x_train, x_test, y_train, y_test
    

In [23]:
def calculate_high_and_low_leverage_tech(data):
            technologies = [feat for feat in data.columns if "WorkedWith" in feat]
            us_data = data[data["Country_for_target_transformation"] =="United States"]
            tech_leverages = {}
            for tech in technologies:
                tech_mean = us_data.groupby(tech).ConvertedComp.mean()
                if len(tech_mean) != 2: continue
                tech_leverage = (tech_mean.loc[1] / tech_mean.loc[0]) -1
                tech_leverages[tech] = tech_leverage
            tech_leverages = pd.Series(tech_leverages).sort_values(ascending=False)

            high_leverage_technologies = tech_leverages[tech_leverages >= tech_leverages.quantile(0.80)].index.tolist()
            low_leverage_technologies = tech_leverages[tech_leverages <= tech_leverages.quantile(0.20)].index.tolist()
            return high_leverage_technologies, low_leverage_technologies

In [24]:
def prepare_data(data=data, 
                 exclude_columns=None, 
                 features_to_use=None, 
                 apply_x_col=None, 
                 columns_to_bin=None,
                 transform_target=False,
                 feat_agg_calculate_chosen_options_ratio=False,
                 feat_agg_worked_with_high_and_low_leverage=False):

    
    if exclude_columns is not None and features_to_use is not None:
        raise Exception("exclude_columns and features_to_use cannot be used at the same time.")

    if exclude_columns is not None:
        data = data.drop(exclude_columns, axis=1)

    if features_to_use is not None:
        data = data[features_to_use + ["revenue"]]
        
    if feat_agg_calculate_chosen_options_ratio:
        def proportion_chosen_options(columns_contains):
            desire_next_year_feats = [feat for feat in data.columns if columns_contains in feat]
            return data[desire_next_year_feats].sum(axis=1) / len(desire_next_year_feats)
        
        data["feat_agg_proportion_chosen_DesireNextYear"] = proportion_chosen_options("DesireNextYear")
        data["feat_agg_proportion_chosen_LanguageDesireNextYear"] = proportion_chosen_options("LanguageDesireNextYear")
        data["feat_agg_proportion_chosen_PlatformDesireNextYear"] = proportion_chosen_options("PlatformDesireNextYear")
        data["feat_agg_proportion_chosen_WebframeDesireNextYear"] = proportion_chosen_options("WebframeDesireNextYear")
        data["feat_agg_proportion_chosen_DatabaseDesireNextYear"] = proportion_chosen_options("DatabaseDesireNextYear")

        data["feat_agg_proportion_chosen_WorkedWith"] = proportion_chosen_options("WorkedWith")
        data["feat_agg_proportion_chosen_LanguageWorkedWith"] = proportion_chosen_options("LanguageWorkedWith")
        data["feat_agg_proportion_chosen_PlatformWorkedWith"] = proportion_chosen_options("PlatformWorkedWith")
        data["feat_agg_proportion_chosen_WebframeWorkedWith"] = proportion_chosen_options("WebframeWorkedWith")
        data["feat_agg_proportion_chosen_DatabaseWorkedWith"] = proportion_chosen_options("DatabaseWorkedWith")
        
        

    x = data.drop('ConvertedComp', axis=1)
    
    y = data['ConvertedComp']

    if apply_x_col is not None:
        x = x.apply(apply_x_col)
    

    x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                        test_size = 0.33, 
                                                        random_state = 3, 
                                                        stratify=x["Country_for_target_transformation"])
    
    if feat_agg_worked_with_high_and_low_leverage:
        
        high_leverage_technologies, low_leverage_technologies = calculate_high_and_low_leverage_tech(pd.concat([x_train, y_train], 
                                                                                                               axis=1))
        def proportion_chosen_given_Tech(technologies):
            return data[technologies].sum(axis=1) / len(technologies)
        
        x_train["feat_agg_high_leverage_tech_worked"] = proportion_chosen_given_Tech(high_leverage_technologies)
        x_train["feat_agg_low_leverage_tech_worked"] = proportion_chosen_given_Tech(low_leverage_technologies)
        
        
        x_test["feat_agg_high_leverage_tech_worked"] = proportion_chosen_given_Tech(high_leverage_technologies)
        x_test["feat_agg_low_leverage_tech_worked"] = proportion_chosen_given_Tech(low_leverage_technologies)
        
        technologies = [feat for feat in data.columns if (("WorkedWith" in feat) or ("DesireNextYear" in feat))]
        x_train = x_train.drop(technologies, axis=1)
        x_test = x_test.drop(technologies, axis=1)
    
    
    if transform_target:
        x_train, x_test, y_train, y_test = target_transformation(x_train, x_test, y_train, y_test)
    
    #Preserved from initial data preparation for target_transformation
    x_train.drop(["Country_for_target_transformation"], axis=1, inplace=True)
    x_test.drop(["Country_for_target_transformation"], axis=1, inplace=True)
        
    

    if columns_to_bin is not None:
        for column in columns_to_bin:
            if column not in x_train.columns: continue
            x_train.loc[:,column], bins_ = pd.qcut(x_train.loc[:,column], q=4,  retbins=True, duplicates="drop")
            x_test.loc[:,column] = pd.cut(x_test.loc[:,column], bins=bins_, )
            
            x_train = pd.get_dummies(x_train)
            x_test = pd.get_dummies(x_test)
    
    return x_train, x_test, y_train, y_test, data

In [25]:
def scale_data(x_train, x_test, scaler=StandardScaler):

    scaler = scaler()
    scaler.fit(x_train)

    x_train_scaled = scaler.transform(x_train)
    x_train_scaled = pd.DataFrame(x_train_scaled, index=x_train.index, columns=x_train.columns)

    x_test_scaled = scaler.transform(x_test)
    x_test_scaled = pd.DataFrame(x_test_scaled, index=x_test.index, columns=x_test.columns)

    x_train_scaled.fillna(x_train_scaled.mean(), inplace=True)
    x_test_scaled.fillna(x_test_scaled.mean(), inplace=True)

    return x_train_scaled, x_test_scaled

In [26]:
def evaluate_regression(y_true_train, y_pred_train, y_true_test, y_pred_test, model_name="", model=None, num_feat=""):
    mape_train = np.abs((y_true_train - y_pred_train) / y_true_train).mean(axis=0)
    mape_test = np.abs((y_true_test - y_pred_test) / y_true_test).mean(axis=0)
    return pd.DataFrame.from_records([[#metrics.mean_squared_error(y_true_train, y_pred_train),
                                        mape_train,
                                       metrics.mean_absolute_error(y_true_train, y_pred_train),
                                       metrics.max_error(y_true_train, y_pred_train),
                                       metrics.r2_score(y_true_train, y_pred_train), 
                                      #metrics.mean_squared_error(y_true_test, y_pred_test),
                                        mape_test,
                                       metrics.mean_absolute_error(y_true_test, y_pred_test),
                                       metrics.max_error(y_true_test, y_pred_test),
                                       metrics.r2_score(y_true_test, y_pred_test),
                                       model,
                                       num_feat]],
                                     
                                     index=[model_name], 
                                     columns=[#'mean_squared_error_train', 
                                             "mape_train",
                                              'mean_absolute_error_train', 
                                              'max_error_train', 
                                              "r2_score_train",
                                              #'mean_squared_error_test', 
                                                 "mape_test",
                                              'mean_absolute_error_test', 
                                              'max_error_test', 
                                              "r2_score_test", 
                                              "model_object", 
                                              "num_feat"])


In [27]:
def run_models(x_train, x_test, y_train, y_test, feture_elimination_num_feat=None):
        results = pd.DataFrame()
        for model in [#linear_model.LinearRegression(), 
                        linear_model.RidgeCV(),
                        linear_model.LassoCV(),
                        #svm.LinearSVR(),
                        #svm.SVR(kernel="rbf",),
                        #svm.SVR(kernel="poly"),
                        #tree.DecisionTreeRegressor(),
                        #RandomForestRegressor()
                     ]:
                
                if feture_elimination_num_feat is not None :
                        try:
                                if feture_elimination_num_feat == "auto":
                                        feture_elimination_num_feat = np.linspace(5,len(x_train.columns),5, dtype=int)

                                for n_features_to_select in feture_elimination_num_feat:
                                        model_ = RFE(estimator=model, n_features_to_select=n_features_to_select)
                                        model_.fit(x_train, y_train)
                                        y_train_pred = model_.predict(x_train)
                                        y_test_pred = model_.predict(x_test)

                                        results = results.append(evaluate_regression(y_train, y_train_pred, y_test, y_test_pred,
                                                                model_name=type(model_).__name__, model=model_, num_feat=n_features_to_select))
                                return results
                        except Exception as e:
                            print(e)
                                
                        
               
                model.fit(x_train, y_train)
                
                y_train_pred = model.predict(x_train)
                y_test_pred = model.predict(x_test)

                results = results.append(evaluate_regression(y_train, y_train_pred, y_test, y_test_pred,
                                        model_name=type(model).__name__, model=model, num_feat=len(x_train.columns) ))
        return results


In [34]:
experiment_list = pd.DataFrame()

def run(exclude_columns=None, 
        features_to_use=None, 
        name="", 
        apply_x_col=None, 
        x_train_=None, 
        y_train_=None, 
        columns_to_bin=None,
        transform_target=False,
        feture_elimination_num_feat=None,
        feat_agg_calculate_chosen_options_ratio=False,
        feat_agg_worked_with_high_and_low_leverage=False):
    global experiment_list
    print(name)

    x_train, x_test, y_train, y_test, data = prepare_data(exclude_columns=exclude_columns, 
                                                        features_to_use=features_to_use, 
                                                        apply_x_col=apply_x_col,
                                                        columns_to_bin=columns_to_bin,
                                                          transform_target=transform_target,
                                                         feat_agg_calculate_chosen_options_ratio=feat_agg_calculate_chosen_options_ratio,
                                                         feat_agg_worked_with_high_and_low_leverage=feat_agg_worked_with_high_and_low_leverage)

    if x_train_ is not None or y_train_ is not None:
        x_train, y_train = x_train_, y_train_

    x_train, x_test = scale_data(x_train, x_test, scaler=StandardScaler)

    results = run_models(x_train, x_test, y_train, y_test, feture_elimination_num_feat=feture_elimination_num_feat)

    best_model = results.sort_values("mean_absolute_error_test").reset_index().loc[[0], ["mean_absolute_error_train", 
                                                                                         "mape_train",
                                                                                         
                                                                                        "mean_absolute_error_test", 
                                                                                         "mape_test",
                                                                                        "index",
                                                                                        "num_feat",
                                                                                        "model_object"]].rename({0:name, "index":"model_name"})
    if experiment_list is not None:
        experiment_list = experiment_list.append(best_model)

    return results.sort_values("mean_absolute_error_test")

In [35]:
results = run(exclude_columns=None, name="Initial Run")

Initial Run


In [36]:
results

,mape_train,mean_absolute_error_train,max_error_train,r2_score_train,mape_test,mean_absolute_error_test,max_error_test,r2_score_test,model_object,num_feat
LassoCV,26.244706,88587.510225,1.848115e+06,0.099391,14.946844,88084.709151,1.913713e+06,0.088789,LassoCV(),619
RidgeCV,27.367006,94006.356781,1.858938e+06,0.115717,17.092047,94422.391113,1.928830e+06,0.083835,"RidgeCV(alphas=array([ 0.1, 1. , 10. ]))",619


In [37]:
results = run(exclude_columns=None, name="With Agg Features",
              feat_agg_calculate_chosen_options_ratio=True,
             feat_agg_worked_with_high_and_low_leverage=True)

With Agg Features


In [38]:
results

,mape_train,mean_absolute_error_train,max_error_train,r2_score_train,mape_test,mean_absolute_error_test,max_error_test,r2_score_test,model_object,num_feat
LassoCV,26.902406,88969.923573,1.851961e+06,0.101010,14.406944,88579.713095,1.918859e+06,0.090167,LassoCV(),409
RidgeCV,28.678171,92252.802754,1.869586e+06,0.107027,15.992825,92340.934351,1.915633e+06,0.086735,"RidgeCV(alphas=array([ 0.1, 1. , 10. ]))",409


In [39]:
experiment_list

,mean_absolute_error_train,mape_train,mean_absolute_error_test,mape_test,index,num_feat,model_object
Initial Run,88587.510225,26.244706,88084.709151,14.946844,LassoCV,619,LassoCV()
With Agg Features,88969.923573,26.902406,88579.713095,14.406944,LassoCV,409,LassoCV()


In [40]:
results = run(name="Initial Run + Feat Selection", feture_elimination_num_feat="auto")
results

Initial Run + Feat Selection


,mape_train,mean_absolute_error_train,max_error_train,r2_score_train,mape_test,mean_absolute_error_test,max_error_test,r2_score_test,model_object,num_feat
RFE,29.302098,92308.424005,1.809362e+06,0.075737,15.479474,91647.810292,1.948562e+06,0.074848,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ...",5
RFE,26.397239,92884.065163,1.848719e+06,0.109514,17.738083,92830.618164,1.933679e+06,0.085593,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ...",158
RFE,27.179648,93883.867614,1.846815e+06,0.115209,17.473572,94311.379066,1.925448e+06,0.083979,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ...",312
RFE,27.367006,94006.356845,1.858938e+06,0.115717,17.092047,94422.391126,1.928830e+06,0.083835,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ...",465
RFE,27.367006,94006.356845,1.858938e+06,0.115717,17.092047,94422.391126,1.928830e+06,0.083835,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ...",619


In [41]:
results = run(exclude_columns=None, name="With Agg Features + Feat Selection",
              feat_agg_calculate_chosen_options_ratio=True,
             feat_agg_worked_with_high_and_low_leverage=True,
             feture_elimination_num_feat="auto")
results

With Agg Features + Feat Selection


,mape_train,mean_absolute_error_train,max_error_train,r2_score_train,mape_test,mean_absolute_error_test,max_error_test,r2_score_test,model_object,num_feat
RFE,28.421521,92424.356652,1.822583e+06,0.076529,14.602195,91580.722159,1.946030e+06,0.075546,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ...",5
RFE,29.196117,91758.557851,1.863467e+06,0.102491,14.323442,91608.922704,1.916926e+06,0.085918,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ...",106
RFE,28.221811,92232.512122,1.868841e+06,0.106921,17.315170,92317.261750,1.916425e+06,0.086803,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ...",207
RFE,28.678171,92252.802754,1.869586e+06,0.107027,15.992825,92340.934351,1.915633e+06,0.086735,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ...",308
RFE,28.678171,92252.802754,1.869586e+06,0.107027,15.992825,92340.934351,1.915633e+06,0.086735,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ...",409


In [42]:
experiment_list

,mean_absolute_error_train,mape_train,mean_absolute_error_test,mape_test,index,num_feat,model_object
Initial Run,88587.510225,26.244706,88084.709151,14.946844,LassoCV,619,LassoCV()
With Agg Features,88969.923573,26.902406,88579.713095,14.406944,LassoCV,409,LassoCV()
Initial Run + Feat Selection,92308.424005,29.302098,91647.810292,15.479474,RFE,5,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ..."
With Agg Features + Feat Selection,92424.356652,28.421521,91580.722159,14.602195,RFE,5,"RFE(estimator=RidgeCV(alphas=array([ 0.1, 1. ..."
